In [ ]:
!nvidia-smi

In [ ]:
import torch

# Check if CUDA is available
print(torch.cuda.is_available())

In [ ]:
from lime import lime_image
from skimage.segmentation import slic
from ultralytics import YOLO
import numpy as np

model_path = "D:/BTXRD-Code/Object-Detection/runs/detect/paper-data/weights/paper-best.pt"
model = YOLO(model_path)
model.conf = 0.01 
print(model.names)

def yolo_predict_fn(images):
    """
    images: list of images (numpy arrays)
    return: list of confidence scores for the target box (e.g., dog box)
    """
    results = []
    for img in images:
        print("Input shape:", img.shape, "dtype:", img.dtype)  # Debug info
        preds = model(img)  # dùng YOLOv8 predict
        probs = np.zeros(len(model.names))
        for pred in preds:
            for box in pred.boxes:
                class_id = int(box.cls[0])
                confidence = box.conf[0]
                probs[class_id] = max(probs[class_id], confidence)
        results.append(probs)

    return np.array(results)

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lime import lime_image
from skimage.segmentation import mark_boundaries
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo

# ---------- Load Detectron2 Model ----------
def load_detectron2_model(config_path="COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml",
                          num_classes=2, weights_path=None, score_thresh=0.5):
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(config_path))
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = num_classes
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = score_thresh
    cfg.MODEL.DEVICE = "cuda"

    if weights_path:
        cfg.MODEL.WEIGHTS = weights_path
    else:
        cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(config_path)

    return DefaultPredictor(cfg)

predictor = load_detectron2_model(num_classes=2, weights_path="./detectron_runs/model_best_6280.pth", score_thresh=0.1)

# ---------- Predictor Function for LIME ----------
def detectron2_predict_fn(images):
    results = []
    for img in images:
        preds = predictor(img)
        instances = preds["instances"].to("cpu")
        scores = np.zeros(2)
        for cls_id, conf in zip(instances.pred_classes.numpy(), instances.scores.numpy()):
            scores[cls_id] = max(scores[cls_id], conf)
        results.append(scores)
    return np.array(results)

In [ ]:
import numpy as np
from PIL import Image
import cv2
import nbimporter
from ensemble_boxes import weighted_boxes_fusion
from ensemblee_wbf import yolo_predict_from_image, detectron2_predict_from_image, load_yolo_model, load_detectron2_model

yolo_model = load_yolo_model("./runs/detect/paper-data/weights/paper-best.pt", confidence_threshold=0.01, iou_threshold=0.6)
detectron_model = load_detectron2_model("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml", 2, "./detectron_runs/model_best_6280.pth", score_thresh=0.01)


def ensemblee_detection_predict_fn(images):
    """
    Predict function for LIME with list of RGB numpy arrays (already resized).
    Returns: Numpy array of shape (N, num_classes) with confidence scores.
    """
    results = []
    num_classes = 2  # Set this to the number of classes in your model

    for img in images:      
        img_h, img_w = img.shape[:2]
        # Run predictions
        yolo_preds = yolo_predict_from_image(yolo_model, img, 608)
        d2_preds = detectron2_predict_from_image(detectron_model, img)

        # Prepare boxes for WBF
        def prepare(preds):
            boxes, scores, labels = [], [], []
            for pred in preds:
                xmin, ymin, width, height = pred["bbox"]
                x1 = xmin / img_w
                y1 = ymin / img_h
                x2 = (xmin + width) / img_w
                y2 = (ymin + height) / img_h
                boxes.append([x1, y1, x2, y2])
                scores.append(pred["score"])
                labels.append(pred["category_id"])
            return boxes, scores, labels

        yolo_boxes, yolo_scores, yolo_labels = prepare(yolo_preds)
        d2_boxes, d2_scores, d2_labels = prepare(d2_preds)

        all_boxes = [yolo_boxes, d2_boxes]
        all_scores = [yolo_scores, d2_scores]
        all_labels = [yolo_labels, d2_labels]

        # Apply Weighted Box Fusion
        boxes_fused, scores_fused, labels_fused = weighted_boxes_fusion(
            all_boxes, all_scores, all_labels, iou_thr=0.55, skip_box_thr=0.0001
        )

        # Build class probabilities
        probs = np.zeros(num_classes)
        for score, label in zip(scores_fused, labels_fused):
            probs[int(label)] = max(probs[int(label)], score)

        results.append(probs)

    return np.array(results)

In [ ]:
import nbimporter
from xai_utils_benmarks import deletion_curve, min_subset, load_yolo_mask, pointing_game, extended_pointing_game

In [ ]:
import cv2
from lime import lime_image
from skimage.segmentation import mark_boundaries
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np


val_folder = r'D:\BTXRD-Dataset\BTXRD-Yolo\val\images'
groundtruth_folder = r'D:\BTXRD-Dataset\BTXRD-Yolo\val\labels'
output_csv_path = 'yolo_lime_results.csv'
output_folder_label_0 = 'LIME_runs\yolo-predict-lime-label-0'
output_folder_label_1 = 'LIME_runs\yolo-predict-lime-label-1'
explainer = lime_image.LimeImageExplainer()

results = []
#  0: B-tumor
#  1: M-tumor
for image_name in os.listdir(val_folder):
    image_path = os.path.join(val_folder, image_name)

    input_image = cv2.imread(image_path)
    original_height, original_width = input_image.shape[:2]
    target_width = 608
    aspect_ratio = original_width / original_height
    new_height = int(target_width / aspect_ratio)
    input_image_resized = cv2.resize(input_image, (target_width, new_height))
       
    # Tạo giải thích cho ảnh
    explanation = explainer.explain_instance(
        input_image_resized,
        yolo_predict_fn,
        top_labels=2,  
        hide_color=0,
        num_samples=1000
    )

    # Hiển thị và lưu lại giải thích cho lớp B-Tumor (label=0)
    temp, mask = explanation.get_image_and_mask(
        label=0, 
        positive_only=True,
        num_features=10,
        hide_rest=False
    )

    segments = explanation.segments  

    # Lặp qua label 0 và 1
    for label, output_folder in zip([0, 1], [output_folder_label_0, output_folder_label_1]):
        weights = dict(explanation.local_exp[label])
        importance_map = np.zeros_like(segments, dtype=float)

        # Gán trọng số dương vào bản đồ importance
        for sp_id, weight in weights.items():
            importance_map[segments == sp_id] = weight if weight > 0 else 0.0

        
        # Chuẩn hóa từ 0 đến 1
        flat = importance_map.flatten().reshape(-1, 1)
        normalized_map = MinMaxScaler().fit_transform(flat).reshape(importance_map.shape)
        normalized_map_resized = cv2.resize(normalized_map, (original_width, original_height))

        print(f"Image: {image_name}, Label: {label}, Max Importance: {np.max(normalized_map_resized)}, Min Importance: {np.min(normalized_map_resized)}")
        print(normalized_map)

        # Vẽ heatmap
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.imshow(cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB))
        heatmap = ax.imshow(normalized_map_resized, cmap='jet', alpha=0.25, vmin=0, vmax=1)
        plt.colorbar(heatmap, ax=ax, fraction=0.046, pad=0.04)
        ax.axis('off')
        ax.set_title(f'LIME Heatmap (Label {label}) - {image_name}')

        # Lưu hình
        heatmap_output_path = os.path.join(output_folder, f"heatmap_label{label}_{image_name}")
        plt.savefig(heatmap_output_path, bbox_inches='tight', pad_inches=0)
        plt.close()

        # Load ground truth mask
        gt_file = os.path.join(groundtruth_folder, os.path.splitext(image_name)[0] + '.txt')
        gt_mask = load_yolo_mask(gt_file, (original_height, original_width), class_filter=label)

        # Tính toán XAI metrics
        sorted_segments = sorted(weights.items(), key=lambda x: -x[1])
        mask_order = [seg_id for seg_id, _ in sorted_segments]
      
        # Tính chỉ số Deletion và D-Deletion
        auc_deletion = deletion_curve(input_image_resized.copy(), segments, mask_order, yolo_predict_fn, label, mode='zero')      

        # Tính chỉ số Min-Subset và D-Min-Subset
        min_subset_score = min_subset(input_image_resized.copy(), segments, weights, yolo_predict_fn, label, threshold=0.9, mode='zero')
       
        # Tính chỉ số Pointing Game và Extended Pointing Game
        pg_score = pointing_game(normalized_map_resized, gt_mask)
        ebpg_score = extended_pointing_game(normalized_map_resized, gt_mask, top_percent=0.01)

        # Ghi lại kết quả
        results.append({
            'image': image_name,
            'label': label,
            'auc_deletion': auc_deletion,
            'auc_d_deletion': 0, # Chưa tính toán
            'min_subset': min_subset_score,
            'd_min_subset': 0, # Chưa tính toán
            'pointing_game': pg_score,
            'extended_pointing_game': ebpg_score,
        })

df = pd.DataFrame(results)
df.to_csv(output_csv_path, index=False)
print(f"Saved metrics to {output_csv_path}")

# Tính trung bình từng chỉ số theo từng label
mean_metrics_per_label = df.groupby('label')[['auc_deletion', 'auc_d_deletion', 'min_subset', 'd_min_subset', 'pointing_game', 'extended_pointing_game']].mean()

# Tạo DataFrame các dòng tổng trung bình cho mỗi label
mean_rows = pd.DataFrame({
    'image': ['mean'] * len(mean_metrics_per_label),
    'label': mean_metrics_per_label.index,
    'auc_deletion': mean_metrics_per_label['auc_deletion'].values,
    'auc_d_deletion': mean_metrics_per_label['auc_d_deletion'].values,
    'min_subset': mean_metrics_per_label['min_subset'].values,
    'd_min_subset': mean_metrics_per_label['d_min_subset'].values,
    'pointing_game': mean_metrics_per_label['pointing_game'].values,
    'extended_pointing_game': mean_metrics_per_label['extended_pointing_game'].values,
})

print("Mean metrics per label:")
print(mean_rows)

# Ghi thêm các dòng tổng trung bình theo label vào file CSV (append, không header)
mean_rows.to_csv(output_csv_path, mode='a', header=False, index=False)

print("Saved mean metrics per label as additional rows in the CSV file.")
print(f"Completed processing.")